# Experiment 007: Hybrid Solution with Kaggle-Compatible Validation

**Goal:** Create a hybrid solution using:
1. Pre-optimized baseline for N values that pass Kaggle-style validation
2. Zaburo fallback for N values with overlaps
3. Apply fractional translation refinement

**Key insight:** Pre-optimized baseline scores ~70.6 but has overlap issues. If we can identify which N values are valid and use Zaburo for the rest, we can get close to 70.6.

In [5]:
import numpy as np
import pandas as pd
from decimal import Decimal, getcontext
from shapely.geometry import Polygon
from shapely import affinity
import copy
import json
import os

# Set high precision for Decimal
getcontext().prec = 28

# Scale factor for integer coordinates (match Kaggle)
SCALE = Decimal('1000000000000000')  # 1e15

print("Setup complete")

Setup complete


In [6]:
# Tree polygon vertices (from getting-started kernel)
TX = [0, 0.125, 0.0625, 0.2, 0.1, 0.35, 0.075, 0.075, -0.075, -0.075, -0.35, -0.1, -0.2, -0.0625, -0.125]
TY = [0.8, 0.5, 0.5, 0.25, 0.25, 0, 0, -0.2, -0.2, 0, 0, 0.25, 0.25, 0.5, 0.5]

class ChristmasTree:
    """Christmas tree with Kaggle-compatible precision."""
    def __init__(self, x, y, angle):
        self.center_x = Decimal(str(x))
        self.center_y = Decimal(str(y))
        self.angle = Decimal(str(angle))
        self._update_polygon()
    
    def _update_polygon(self):
        """Update polygon vertices based on position and angle."""
        # Create base polygon
        coords = list(zip(TX, TY))
        poly = Polygon(coords)
        # Rotate and translate
        poly = affinity.rotate(poly, float(self.angle), origin=(0, 0))
        poly = affinity.translate(poly, float(self.center_x), float(self.center_y))
        self.polygon = poly
        self.vertices = list(poly.exterior.coords)[:-1]  # Remove duplicate last point
    
    def get_integer_polygon(self):
        """Get polygon with integer coordinates for Kaggle-style validation."""
        int_coords = []
        for x, y in self.vertices:
            xi = int(Decimal(str(x)) * SCALE)
            yi = int(Decimal(str(y)) * SCALE)
            int_coords.append((xi, yi))
        return Polygon(int_coords)
    
    def clone(self):
        return ChristmasTree(float(self.center_x), float(self.center_y), float(self.angle))

print("ChristmasTree class defined")

ChristmasTree class defined


In [7]:
def check_overlap_kaggle_style(trees):
    """Check overlaps using integer coordinates like Kaggle does."""
    polygons = [t.get_integer_polygon() for t in trees]
    
    for i in range(len(polygons)):
        for j in range(i+1, len(polygons)):
            if polygons[i].intersects(polygons[j]):
                # Check if it's just touching (allowed) or actual overlap (not allowed)
                if not polygons[i].touches(polygons[j]):
                    # Check intersection area
                    try:
                        intersection = polygons[i].intersection(polygons[j])
                        if intersection.area > 0:
                            return True, (i, j), intersection.area
                    except:
                        return True, (i, j), -1
    return False, None, 0

def calculate_bbox_side(trees):
    """Calculate bounding box side length."""
    all_x = []
    all_y = []
    for tree in trees:
        for x, y in tree.vertices:
            all_x.append(x)
            all_y.append(y)
    
    min_x, max_x = min(all_x), max(all_x)
    min_y, max_y = min(all_y), max(all_y)
    
    return max(max_x - min_x, max_y - min_y)

def calculate_score(trees, n):
    """Calculate score contribution for n trees."""
    side = calculate_bbox_side(trees)
    return (side ** 2) / n

print("Validation functions defined")

Validation functions defined


In [8]:
# Load pre-optimized baseline
baseline_path = '/home/nonroot/snapshots/santa-2025/21329067673/submission/submission.csv'
baseline_df = pd.read_csv(baseline_path)
print(f"Loaded baseline with {len(baseline_df)} rows")
print(baseline_df.head())

# Parse the 'id' column to get n and i
# Format: "001_0" means N=1, tree index 0
def parse_id(id_str):
    parts = id_str.split('_')
    n = int(parts[0])
    i = int(parts[1])
    return n, i

# Parse the 's' prefix from coordinate values
def parse_coord(val):
    if isinstance(val, str) and val.startswith('s'):
        return float(val[1:])
    return float(val)

# Parse baseline into trees by N
baseline_trees = {}
for _, row in baseline_df.iterrows():
    n, i = parse_id(row['id'])
    x = parse_coord(row['x'])
    y = parse_coord(row['y'])
    deg = parse_coord(row['deg'])
    
    if n not in baseline_trees:
        baseline_trees[n] = []
    
    tree = ChristmasTree(x, y, deg)
    baseline_trees[n].append(tree)

print(f"Parsed {len(baseline_trees)} N values")
print(f"N=1 has {len(baseline_trees[1])} trees, N=200 has {len(baseline_trees[200])} trees")

Loaded baseline with 20100 rows
      id                       x                       y  \
0  001_0                    s0.0                    s0.0   
1  002_0   s0.154097069621355887  s-0.038540742694794648   
2  002_1  s-0.154097069621372845  s-0.561459257305224058   
3  003_0      s1.123655816140301      s0.781101815992563   
4  003_1       s1.23405569584216      s1.275999500663759   

                       deg  
0                    s45.0  
1  s203.629377730656841550  
2   s23.629377730656791812  
3        s111.125132292893  
4         s66.370622269343  


Parsed 200 N values
N=1 has 1 trees, N=200 has 200 trees


In [9]:
# Load Zaburo solution (guaranteed valid)
zaburo_path = '/home/code/experiments/005_zaburo_rowbased/submission.csv'
zaburo_df = pd.read_csv(zaburo_path)
print(f"Loaded Zaburo with {len(zaburo_df)} rows")
print(zaburo_df.head())

# Check format of Zaburo
if 'n' in zaburo_df.columns:
    # Standard format with n, i columns
    zaburo_trees = {}
    for n in range(1, 201):
        n_df = zaburo_df[zaburo_df['n'] == n].sort_values('i')
        trees = []
        for _, row in n_df.iterrows():
            tree = ChristmasTree(row['x'], row['y'], row['deg'])
            trees.append(tree)
        zaburo_trees[n] = trees
else:
    # Same format as baseline with 'id' column
    zaburo_trees = {}
    for _, row in zaburo_df.iterrows():
        n, i = parse_id(row['id'])
        x = parse_coord(row['x'])
        y = parse_coord(row['y'])
        deg = parse_coord(row['deg'])
        
        if n not in zaburo_trees:
            zaburo_trees[n] = []
        
        tree = ChristmasTree(x, y, deg)
        zaburo_trees[n].append(tree)

print(f"Parsed {len(zaburo_trees)} N values")

Loaded Zaburo with 20100 rows
      id      x     y     deg
0  001_0   s0.0  s0.0   s45.0
1  002_0   s0.0  s0.0    s0.0
2  002_1  s0.35  s0.8  s180.0
3  003_0   s0.0  s0.0    s0.0
4  003_1   s0.7  s0.0    s0.0


Parsed 200 N values


In [10]:
# Check each N value in baseline for overlaps using Kaggle-style validation
print("Checking baseline for overlaps with Kaggle-style validation...")

overlapping_n = []
valid_n = []

for n in range(1, 201):
    trees = baseline_trees[n]
    has_overlap, pair, area = check_overlap_kaggle_style(trees)
    
    if has_overlap:
        overlapping_n.append(n)
        if n <= 20 or len(overlapping_n) <= 10:
            print(f"N={n}: OVERLAP between trees {pair}, area={area}")
    else:
        valid_n.append(n)

print(f"\nTotal overlapping N values: {len(overlapping_n)}")
print(f"Total valid N values: {len(valid_n)}")
print(f"\nOverlapping N values: {overlapping_n[:30]}..." if len(overlapping_n) > 30 else f"\nOverlapping N values: {overlapping_n}")

Checking baseline for overlaps with Kaggle-style validation...
N=4: OVERLAP between trees (1, 2), area=0.67034912109375
N=5: OVERLAP between trees (0, 3), area=0.0732421875
N=8: OVERLAP between trees (0, 5), area=0.005859375
N=12: OVERLAP between trees (0, 8), area=2.4375
N=16: OVERLAP between trees (4, 7), area=0.01953125
N=20: OVERLAP between trees (4, 18), area=0.0625
N=26: OVERLAP between trees (4, 11), area=3.916015625
N=33: OVERLAP between trees (0, 3), area=0.027618408203125


N=55: OVERLAP between trees (9, 24), area=0.0078125
N=62: OVERLAP between trees (18, 52), area=0.015625



Total overlapping N values: 24
Total valid N values: 176

Overlapping N values: [4, 5, 8, 12, 16, 20, 26, 33, 55, 62, 70, 84, 99, 102, 103, 112, 127, 134, 135, 138, 151, 154, 161, 166]


In [11]:
# Calculate scores for baseline valid N values vs Zaburo
print("\nComparing scores for valid baseline N values vs Zaburo:")

baseline_valid_total = 0
zaburo_total = 0

for n in range(1, 201):
    baseline_score = calculate_score(baseline_trees[n], n)
    zaburo_score = calculate_score(zaburo_trees[n], n)
    
    if n in valid_n:
        baseline_valid_total += baseline_score
    zaburo_total += zaburo_score

print(f"Baseline (valid N only): {baseline_valid_total:.6f}")
print(f"Zaburo total: {zaburo_total:.6f}")

# Calculate what hybrid would score
hybrid_total = 0
for n in range(1, 201):
    if n in valid_n:
        hybrid_total += calculate_score(baseline_trees[n], n)
    else:
        hybrid_total += calculate_score(zaburo_trees[n], n)

print(f"\nHybrid (baseline valid + Zaburo fallback): {hybrid_total:.6f}")


Comparing scores for valid baseline N values vs Zaburo:
Baseline (valid N only): 62.013886
Zaburo total: 87.991248

Hybrid (baseline valid + Zaburo fallback): 73.265426


In [12]:
# Create hybrid solution
print("Creating hybrid solution...")

hybrid_trees = {}
for n in range(1, 201):
    if n in valid_n:
        # Use baseline (better score)
        hybrid_trees[n] = [t.clone() for t in baseline_trees[n]]
    else:
        # Use Zaburo (guaranteed valid)
        hybrid_trees[n] = [t.clone() for t in zaburo_trees[n]]

# Verify hybrid has no overlaps
print("\nVerifying hybrid solution has no overlaps...")
hybrid_overlaps = []
for n in range(1, 201):
    has_overlap, pair, area = check_overlap_kaggle_style(hybrid_trees[n])
    if has_overlap:
        hybrid_overlaps.append(n)
        print(f"N={n}: OVERLAP in hybrid!")

print(f"\nHybrid overlapping N values: {len(hybrid_overlaps)}")
if len(hybrid_overlaps) == 0:
    print("✅ Hybrid solution is VALID (0 overlaps)")

Creating hybrid solution...



Verifying hybrid solution has no overlaps...
N=8: OVERLAP in hybrid!
N=12: OVERLAP in hybrid!
N=16: OVERLAP in hybrid!
N=20: OVERLAP in hybrid!
N=26: OVERLAP in hybrid!
N=33: OVERLAP in hybrid!
N=55: OVERLAP in hybrid!


N=62: OVERLAP in hybrid!
N=70: OVERLAP in hybrid!


N=84: OVERLAP in hybrid!


N=99: OVERLAP in hybrid!
N=102: OVERLAP in hybrid!
N=103: OVERLAP in hybrid!


...

N=134: OVERLAP in hybrid!
N=135: OVERLAP in hybrid!
N=138: OVERLAP in hybrid!


N=151: OVERLAP in hybrid!
N=154: OVERLAP in hybrid!


N=161: OVERLAP in hybrid!


N=166: OVERLAP in hybrid!



Hybrid overlapping N values: 22


In [13]:
# Debug: Check if Zaburo has overlaps for the problematic N values
print("Checking Zaburo for overlaps in problematic N values...")

for n in [4, 5, 8, 12, 16, 20]:
    trees = zaburo_trees[n]
    has_overlap, pair, area = check_overlap_kaggle_style(trees)
    print(f"Zaburo N={n}: overlap={has_overlap}, pair={pair}, area={area}")

# Also check what's in valid_n vs overlapping_n
print(f"\nN=4 in valid_n: {4 in valid_n}")
print(f"N=4 in overlapping_n: {4 in overlapping_n}")
print(f"N=5 in valid_n: {5 in valid_n}")
print(f"N=5 in overlapping_n: {5 in overlapping_n}")

Checking Zaburo for overlaps in problematic N values...
Zaburo N=4: overlap=False, pair=None, area=0
Zaburo N=5: overlap=False, pair=None, area=0
Zaburo N=8: overlap=True, pair=(2, 4), area=0.4789915966386554
Zaburo N=12: overlap=True, pair=(2, 4), area=0.4789915966386554
Zaburo N=16: overlap=True, pair=(2, 5), area=0.4789915966386554
Zaburo N=20: overlap=True, pair=(2, 5), area=0.4789915966386554

N=4 in valid_n: False
N=4 in overlapping_n: True
N=5 in valid_n: False
N=5 in overlapping_n: True


In [15]:
# Debug: Check Zaburo N=8 tree positions
print("Zaburo N=8 tree positions:")
for i, tree in enumerate(zaburo_trees[8]):
    print(f"  Tree {i}: x={float(tree.center_x):.4f}, y={float(tree.center_y):.4f}, angle={float(tree.angle):.1f}")

# Check the actual polygons
print("\nChecking polygon intersection for trees 2 and 4:")
poly2 = zaburo_trees[8][2].get_integer_polygon()
poly4 = zaburo_trees[8][4].get_integer_polygon()
print(f"  Poly2 bounds: {poly2.bounds}")
print(f"  Poly4 bounds: {poly4.bounds}")
print(f"  Intersects: {poly2.intersects(poly4)}")
print(f"  Touches: {poly2.touches(poly4)}")

# Check with float polygons
poly2_float = zaburo_trees[8][2].polygon
poly4_float = zaburo_trees[8][4].polygon
print(f"\nFloat polygon check:")
print(f"  Intersects: {poly2_float.intersects(poly4_float)}")
print(f"  Touches: {poly2_float.touches(poly4_float)}")
if poly2_float.intersects(poly4_float) and not poly2_float.touches(poly4_float):
    intersection = poly2_float.intersection(poly4_float)
    print(f"  Intersection area: {intersection.area}")

Zaburo N=8 tree positions:
  Tree 0: x=0.0000, y=0.0000, angle=0.0
  Tree 1: x=0.7000, y=0.0000, angle=0.0
  Tree 2: x=1.4000, y=0.0000, angle=0.0
  Tree 3: x=0.3500, y=0.8000, angle=180.0
  Tree 4: x=1.0500, y=0.8000, angle=180.0
  Tree 5: x=0.0000, y=1.0000, angle=0.0
  Tree 6: x=0.7000, y=1.0000, angle=0.0
  Tree 7: x=1.4000, y=1.0000, angle=0.0

Checking polygon intersection for trees 2 and 4:
  Poly2 bounds: (1049999999999999.0, -200000000000000.0, 1750000000000000.0, 800000000000000.0)
  Poly4 bounds: (700000000000000.0, 0.0, 1400000000000000.0, 1000000000000000.0)
  Intersects: True
  Touches: False

Float polygon check:
  Intersects: True
  Touches: False
  Intersection area: 1.7401343497522318e-32


In [16]:
# Updated validation with tolerance for numerical noise
def check_overlap_with_tolerance(trees, area_threshold=1e-10):
    """Check overlaps with tolerance for numerical noise."""
    polygons = [t.polygon for t in trees]  # Use float polygons
    
    for i in range(len(polygons)):
        for j in range(i+1, len(polygons)):
            if polygons[i].intersects(polygons[j]):
                if not polygons[i].touches(polygons[j]):
                    try:
                        intersection = polygons[i].intersection(polygons[j])
                        if intersection.area > area_threshold:
                            return True, (i, j), intersection.area
                    except:
                        pass
    return False, None, 0

# Re-check Zaburo with tolerance
print("Re-checking Zaburo with tolerance...")
for n in [4, 5, 8, 12, 16, 20]:
    trees = zaburo_trees[n]
    has_overlap, pair, area = check_overlap_with_tolerance(trees)
    print(f"Zaburo N={n}: overlap={has_overlap}, pair={pair}, area={area}")

# Re-check baseline with tolerance
print("\nRe-checking baseline with tolerance...")
overlapping_n_v2 = []
valid_n_v2 = []

for n in range(1, 201):
    trees = baseline_trees[n]
    has_overlap, pair, area = check_overlap_with_tolerance(trees)
    
    if has_overlap:
        overlapping_n_v2.append(n)
    else:
        valid_n_v2.append(n)

print(f"\nTotal overlapping N values (with tolerance): {len(overlapping_n_v2)}")
print(f"Total valid N values (with tolerance): {len(valid_n_v2)}")
print(f"Overlapping N values: {overlapping_n_v2}")

Re-checking Zaburo with tolerance...
Zaburo N=4: overlap=False, pair=None, area=0
Zaburo N=5: overlap=False, pair=None, area=0
Zaburo N=8: overlap=False, pair=None, area=0
Zaburo N=12: overlap=False, pair=None, area=0
Zaburo N=16: overlap=False, pair=None, area=0
Zaburo N=20: overlap=False, pair=None, area=0

Re-checking baseline with tolerance...



Total overlapping N values (with tolerance): 0
Total valid N values (with tolerance): 200
Overlapping N values: []


In [17]:
# Check N=2 specifically - Kaggle said "Overlapping trees in group 002"
print("Checking N=2 in detail...")
trees_n2 = baseline_trees[2]
print(f"N=2 has {len(trees_n2)} trees")

for i, tree in enumerate(trees_n2):
    print(f"  Tree {i}: x={float(tree.center_x):.15f}, y={float(tree.center_y):.15f}, angle={float(tree.angle):.15f}")

# Check intersection with different thresholds
poly0 = trees_n2[0].polygon
poly1 = trees_n2[1].polygon

print(f"\nPolygon 0 bounds: {poly0.bounds}")
print(f"Polygon 1 bounds: {poly1.bounds}")
print(f"Intersects: {poly0.intersects(poly1)}")
print(f"Touches: {poly0.touches(poly1)}")

if poly0.intersects(poly1):
    intersection = poly0.intersection(poly1)
    print(f"Intersection type: {intersection.geom_type}")
    print(f"Intersection area: {intersection.area}")
    if hasattr(intersection, 'length'):
        print(f"Intersection length: {intersection.length}")

# Also check with integer coordinates
int_poly0 = trees_n2[0].get_integer_polygon()
int_poly1 = trees_n2[1].get_integer_polygon()
print(f"\nInteger polygon check:")
print(f"Intersects: {int_poly0.intersects(int_poly1)}")
print(f"Touches: {int_poly0.touches(int_poly1)}")
if int_poly0.intersects(int_poly1) and not int_poly0.touches(int_poly1):
    int_intersection = int_poly0.intersection(int_poly1)
    print(f"Intersection area: {int_intersection.area}")

Checking N=2 in detail...
N=2 has 2 trees
  Tree 0: x=0.154097069621356, y=-0.038540742694795, angle=203.629377730656842
  Tree 1: x=-0.154097069621373, y=-0.561459257305224, angle=23.629377730656792

Polygon 0 bounds: (-0.16655799759441026, -0.771466610616546, 0.47475213683712214, 0.17475213683712126)
Polygon 1 bounds: (-0.4747521368371391, -0.77475213683714, 0.1665579975943934, 0.17146661061652746)
Intersects: False
Touches: False

Integer polygon check:
Intersects: False
Touches: False


In [18]:
# Check the raw submission file for N=2
print("Raw submission data for N=2:")
n2_rows = baseline_df[baseline_df['id'].str.startswith('002_')]
print(n2_rows)

# Check if there's a precision issue in parsing
print("\nParsing check:")
for _, row in n2_rows.iterrows():
    x_str = row['x']
    y_str = row['y']
    deg_str = row['deg']
    print(f"  Raw: x={x_str}, y={y_str}, deg={deg_str}")
    print(f"  Parsed: x={parse_coord(x_str)}, y={parse_coord(y_str)}, deg={parse_coord(deg_str)}")

Raw submission data for N=2:
      id                       x                       y  \
1  002_0   s0.154097069621355887  s-0.038540742694794648   
2  002_1  s-0.154097069621372845  s-0.561459257305224058   

                       deg  
1  s203.629377730656841550  
2   s23.629377730656791812  

Parsing check:
  Raw: x=s0.154097069621355887, y=s-0.038540742694794648, deg=s203.629377730656841550
  Parsed: x=0.1540970696213559, y=-0.03854074269479465, deg=203.62937773065684
  Raw: x=s-0.154097069621372845, y=s-0.561459257305224058, deg=s23.629377730656791812
  Parsed: x=-0.15409706962137285, y=-0.5614592573052241, deg=23.629377730656792


In [ ]:
# Let's take a different approach - use the SA-optimized solution from exp_006 as starting point
# This was ACCEPTED by Kaggle with LB=87.81

sa_path = '/home/code/experiments/006_sa_from_scratch/submission.csv'
sa_df = pd.read_csv(sa_path)
print(f"Loaded SA solution with {len(sa_df)} rows")
print(sa_df.head())

# Parse SA solution
sa_trees = {}
for _, row in sa_df.iterrows():
    n, i = parse_id(row['id'])
    x = parse_coord(row['x'])
    y = parse_coord(row['y'])
    deg = parse_coord(row['deg'])
    
    if n not in sa_trees:
        sa_trees[n] = []
    
    tree = ChristmasTree(x, y, deg)
    sa_trees[n].append(tree)

print(f"Parsed {len(sa_trees)} N values")

# Calculate SA score
sa_score = 0
for n in range(1, 201):
    sa_score += calculate_score(sa_trees[n], n)
print(f"SA total score: {sa_score:.6f}")

In [ ]:
# Calculate final hybrid score
hybrid_score = 0
per_n_scores = {}

for n in range(1, 201):
    score = calculate_score(hybrid_trees[n], n)
    per_n_scores[n] = score
    hybrid_score += score

print(f"Hybrid total score: {hybrid_score:.6f}")
print(f"\nTop 10 score contributors:")
sorted_scores = sorted(per_n_scores.items(), key=lambda x: x[1], reverse=True)
for n, score in sorted_scores[:10]:
    source = 'baseline' if n in valid_n else 'zaburo'
    print(f"  N={n}: {score:.6f} ({source})")

In [ ]:
# Now apply fractional translation refinement to improve the hybrid
print("Applying fractional translation refinement...")

frac_steps = [0.01, 0.005, 0.002, 0.001, 0.0005, 0.0002, 0.0001]
directions = [(0, 1), (0, -1), (1, 0), (-1, 0), (1, 1), (1, -1), (-1, 1), (-1, -1)]

def fractional_translation(trees, n, max_iter=50):
    """Apply fractional translation refinement."""
    best_trees = [t.clone() for t in trees]
    best_score = calculate_score(best_trees, n)
    
    for iteration in range(max_iter):
        improved = False
        for i in range(len(best_trees)):
            for step in frac_steps:
                for dx, dy in directions:
                    # Save old position
                    old_x = best_trees[i].center_x
                    old_y = best_trees[i].center_y
                    
                    # Try move
                    best_trees[i].center_x = old_x + Decimal(str(dx * step))
                    best_trees[i].center_y = old_y + Decimal(str(dy * step))
                    best_trees[i]._update_polygon()
                    
                    # Check if valid and better
                    has_overlap, _, _ = check_overlap_kaggle_style(best_trees)
                    if not has_overlap:
                        new_score = calculate_score(best_trees, n)
                        if new_score < best_score - 1e-10:
                            best_score = new_score
                            improved = True
                            continue  # Keep the move
                    
                    # Revert
                    best_trees[i].center_x = old_x
                    best_trees[i].center_y = old_y
                    best_trees[i]._update_polygon()
        
        if not improved:
            break
    
    return best_trees, best_score

print("Fractional translation function defined")

In [ ]:
# Apply fractional translation to small N values (highest impact)
print("Applying fractional translation to N=1-20...")

improved_trees = {}
total_improvement = 0

for n in range(1, 21):
    original_score = calculate_score(hybrid_trees[n], n)
    improved, new_score = fractional_translation(hybrid_trees[n], n, max_iter=30)
    improved_trees[n] = improved
    
    improvement = original_score - new_score
    total_improvement += improvement
    
    if improvement > 1e-8:
        print(f"N={n}: {original_score:.6f} -> {new_score:.6f} (improved by {improvement:.6f})")
    else:
        print(f"N={n}: {original_score:.6f} (no improvement)")

print(f"\nTotal improvement from fractional translation: {total_improvement:.6f}")

In [ ]:
# Update hybrid with improved trees
for n in range(1, 21):
    hybrid_trees[n] = improved_trees[n]

# Calculate final score
final_score = 0
for n in range(1, 201):
    final_score += calculate_score(hybrid_trees[n], n)

print(f"Final hybrid score: {final_score:.6f}")
print(f"Improvement over pure Zaburo (87.99): {87.99 - final_score:.6f}")
print(f"Gap to target (68.89): {final_score - 68.89:.6f}")

In [ ]:
# Final validation - check ALL N values for overlaps
print("Final validation - checking all N values...")

final_overlaps = []
for n in range(1, 201):
    has_overlap, pair, area = check_overlap_kaggle_style(hybrid_trees[n])
    if has_overlap:
        final_overlaps.append((n, pair, area))
        print(f"N={n}: OVERLAP between trees {pair}")

if len(final_overlaps) == 0:
    print("\n✅ FINAL VALIDATION PASSED: 0 overlapping N values")
else:
    print(f"\n❌ VALIDATION FAILED: {len(final_overlaps)} overlapping N values")

In [ ]:
# Save submission
rows = []
for n in range(1, 201):
    for i, tree in enumerate(hybrid_trees[n]):
        rows.append({
            'n': n,
            'i': i,
            'x': float(tree.center_x),
            'y': float(tree.center_y),
            'deg': float(tree.angle)
        })

submission_df = pd.DataFrame(rows)
submission_df.to_csv('/home/code/experiments/007_hybrid_kaggle_validation/submission.csv', index=False)
submission_df.to_csv('/home/submission/submission.csv', index=False)

print(f"Saved submission with {len(submission_df)} rows")
print(submission_df.head())

In [ ]:
# Save metrics
metrics = {
    'cv_score': final_score,
    'hybrid_score_before_frac': hybrid_score,
    'improvement_from_frac': total_improvement,
    'valid_n_count': len(valid_n),
    'overlapping_n_count': len(overlapping_n),
    'final_overlaps': len(final_overlaps)
}

with open('/home/code/experiments/007_hybrid_kaggle_validation/metrics.json', 'w') as f:
    json.dump(metrics, f, indent=2)

print("Metrics saved:")
print(json.dumps(metrics, indent=2))